In [29]:
import sys
import os

#add paths
sys.path.append(os.path.realpath('../'))
import hybrid_lcaio
import run_hybrid_MC as mc

# the following needs to be set in order for the ray workers to also recognise the path above
os.environ['PYTHONPATH'] = os.path.realpath('../')


# from importlib import reload
# reload(mc)
# import gc
# import psutil


<module 'run_hybrid_MC' from '/home/jakobs/Documents/IndEcol/OASES/Geographic_uncertainty/scripts/run_hybrid_MC.py'>

In [17]:
# Define MC parameters
MC_par_dic = {
    'N_runs' : 10000,                                                                    
    'constant_price' : False,                                                           
    'use_ecoinvent_price' : False, # this overrides the constant price flag             
    'constant_region_var_price' : False, # this overrides the above two flags                                                                
    'FU' : None,
    'indices' : [5,31] # these are the indices of the calculated impacts
    }

# Define paths, etc
double_counting_method = 'binary'                                                
exiobase_version = '3_8_2'                                                       
ecoinvent_version = '3_8'
N_cores = 1
Database_dir = '/home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/'
lcaio_object_path = os.path.join(Database_dir,'plcaio_object_STAM_saved_both_strategies_True_ecoinvent_3_8_EXIOBASE_3_8_2.pickle')
directory_baci_price_data = os.path.join(Database_dir,'price_data_per_region/2015/')
# C_io_path = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase3_6/Characterization_EB36.xlsx'
price_data_feather_file = os.path.join(Database_dir,'price_dataframe_ecoinvent_3_8_2015_regionally_aggregated.ftr')
results_file_path = '/home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/Impacts_Hybrid_MC_ecoinvent_3_8_exiobase_3_8_2_double_counting_correction_binary.hdf5'
hdb_object_file_name = os.path.join(Database_dir,'HDB_ecoinvent_{}_exiobase_{}.pickle'.format(ecoinvent_version, exiobase_version))

In [3]:
%%time
# Read in data and initialise/read in hybrid object
hdb, hdb_object_file_name = mc.initiate_hdb(exiobase_version=exiobase_version,
                ecoinvent_version=ecoinvent_version,
                Database_dir=Database_dir,
                lcaio_object_path=lcaio_object_path,
                hdb_object_file_name=hdb_object_file_name
                )

loading Hybrid Database Object...
CPU times: user 1.25 s, sys: 4.73 s, total: 5.98 s
Wall time: 6.83 s


In [4]:
%%time
# Prepare MC, building necessary dictionaries, loading some more data if necessary
mc.prepare_calculations(hdb,
                        MC_par_dic=MC_par_dic,
                        double_counting_method=double_counting_method,
                        # results_file_path=results_file_path,
                        directory_baci_price_data=directory_baci_price_data,
                        price_data_feather_file=price_data_feather_file
                        )

Results will be saved in /home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/Impacts_Hybrid_MC_ecoinvent_3_8_exiobase_3_8_2_double_counting_correction_binary.hdf5
You are about to overwrite the file /home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/Impacts_Hybrid_MC_ecoinvent_3_8_exiobase_3_8_2_double_counting_correction_binary.hdf5


Do you want to do this? Y/N   y


Will overwrite file /home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/Impacts_Hybrid_MC_ecoinvent_3_8_exiobase_3_8_2_double_counting_correction_binary.hdf5
Building price dictionary...
Initialising the results hdf5 file...
CPU times: user 5.83 s, sys: 1.04 s, total: 6.86 s
Wall time: 9.44 s


In [8]:
psutil.virtual_memory()

svmem(total=25091981312, available=14966988800, percent=40.4, used=8992382976, free=6622871552, active=13486682112, inactive=4230737920, buffers=282222592, cached=9194504192, shared=720424960, slab=427941888)

In [30]:
%%time
# Now run MC!
mc.run_monte_carlo(hdb,
                   MC_par_dic=MC_par_dic,
                   results_file_path=results_file_path,
                   hdb_object_file_name=hdb_object_file_name,
                   N_cores=N_cores
                   )

Run 1 of 10. Mon Nov 14 15:50:50 2022
Run 10 of 10. Mon Nov 14 15:51:51 2022
Done calculating results
Elapsed wall time for Calculating 10 runs: 0.0 hours, 1.0, minutes, and 8 seconds.
saving hybrid db as /home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/HDB_ecoinvent_3_8_exiobase_3_8_2.pickle...
writing meta data to /home/jakobs/Documents/IndEcol/OASES/EuropeanConsumptionFootprint/Databases/Impacts_Hybrid_MC_ecoinvent_3_8_exiobase_3_8_2_double_counting_correction_binary.txt
Done! Mon Nov 14 15:52:21 2022
CPU times: user 59.2 s, sys: 21.6 s, total: 1min 20s
Wall time: 1min 30s


0